In [1]:
# net level
import pNN_NetLevel_LNC_variation as pNN

# packages
import os
import math
import torch
import random
import pickle
import calendar
import numpy as np
import config
import training_together_variation as training 
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
os.environ['KMP_DUPLICATE_LIB_OK']='True'
torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
datasets = os.listdir('./dataset')
datasets = [f for f in datasets if (f.startswith('Dataset') and f.endswith('.p'))] 
#datasets

In [3]:

result = torch.zeros((13, 4, 10))
lrs = [0.05, 0.01, 0.005, 0.001]
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(lrs)):
        lr = lrs[j]
        
        for n in range(10):
            seed = n      
            setup = f'dataset_{data_name}_epsilon_0.1_lr_{lr}_seed_{seed}'
            print(setup)
            
            if os.path.exists(f'./results_variation/pNN_{setup}'):
                print('File exists, pass.')
            else:
                torch.manual_seed(seed)
                model = pNN.pNN([N_feature, 3, N_class], 10, 0.1)
    
                optimizer = torch.optim.Adam([{'params':model.GetParam('theta_'),'lr':1e-1},
                                  {'params':model.GetParam('eta_'), 'lr':lr},
                                  {'params':model.GetParam('inv_'),'lr':lr}])
    
    
                NN, acc_valid = training.training_pNN(model, train_loader, valid_loader, optimizer, pNN.lossfunction)
                result[i,j,n] = acc_valid
                torch.save(NN, f'./results_variation/pNN_{setup}')

a = {'acc_valid': result}
with open('acc_valid_variation.txt', 'wb') as file:
    pickle.dump(a, file)

dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_0
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_1
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_2
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_3
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_4
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_5
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_6
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_7
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_8
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.05_seed_9
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.01_seed_0
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.01_seed_1
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.01_seed_2
File exists, pass.
dataset_acuteinflammation_epsilon_0.1_lr_0.01_seed_3
File exists

## valid

In [13]:

result = torch.zeros((13, 6, 10))
lrs = [0.0001, 0.0005,0.001, 0.005, 0.01,0.05]
#lrs = [ 0.0005,0.001, 0.005, 0.01,0.05]
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(lrs)):
        lr = lrs[j]
        
        for n in range(10):
            seed = n
            
            setup = f'dataset_{data_name}_lr_{lr}_seed_{seed}_learnable1'
            model = torch.load(f'./results_variation/pNN_{setup}')
            #model.SetParameter('N', 100)
            
            
            with torch.no_grad():
                for X_valid, y_valid in valid_loader:
                    #print(setup)
                    prediction_valid = model(X_valid)
                    #acc_valid = lossfunction(prediction_valid, y_valid)
                    yhat_valid = torch.argmax(prediction_valid.data, 2)
                
                    yy_valid = y_valid.repeat(prediction_valid.shape[0], 1)
                    valid_correct = torch.sum(yhat_valid == yy_valid.data)
                    acc_valid = valid_correct / (y_valid.numel() * prediction_valid.shape[0])
                    result[i,j,n] = acc_valid
                

#a = {'acc_valid': result}
#with open('acc_valid_variation.txt', 'wb') as file:
    #pickle.dump(a, file)

In [14]:
result1=result.mean(dim = 2)
result1.shape, result1

(torch.Size([13, 6]),
 tensor([[1.0000, 1.0000, 0.9217, 1.0000, 1.0000, 1.0000],
         [0.9435, 0.9645, 0.9597, 0.9758, 0.9685, 0.9669],
         [0.9698, 0.9647, 0.9604, 0.9698, 0.9554, 0.9662],
         [0.8851, 0.8542, 0.8533, 0.7601, 0.7965, 0.8123],
         [0.8706, 0.8255, 0.8059, 0.8190, 0.8412, 0.8092],
         [0.9242, 0.8660, 0.8425, 0.8033, 0.8660, 0.8438],
         [0.9862, 0.9862, 0.9552, 0.9759, 0.9103, 0.9414],
         [0.8330, 0.7911, 0.6660, 0.8267, 0.8183, 0.8251],
         [0.4741, 0.4778, 0.4240, 0.2089, 0.3662, 0.3695],
         [0.9561, 0.9195, 0.9146, 0.9317, 0.8951, 0.8976],
         [0.9089, 0.8157, 0.7948, 0.8194, 0.7571, 0.7691],
         [0.8672, 0.8230, 0.8164, 0.7262, 0.7180, 0.8541],
         [0.8590, 0.8311, 0.8426, 0.7443, 0.8410, 0.8279]]))

In [15]:
result2 = result1.mean(dim = 0)

In [80]:
result2

tensor([0.8829, 0.8553, 0.8275, 0.8124, 0.8257, 0.8372])

In [16]:
mean= result.std(dim=2)
mean.shape, mean

(torch.Size([13, 6]),
 tensor([[0.0000, 0.0000, 0.1650, 0.0000, 0.0000, 0.0000],
         [0.0344, 0.0068, 0.0000, 0.0120, 0.0111, 0.0080],
         [0.0046, 0.0086, 0.0051, 0.0057, 0.0095, 0.0059],
         [0.0502, 0.0250, 0.0406, 0.0327, 0.0472, 0.0534],
         [0.0192, 0.0260, 0.1257, 0.0369, 0.0392, 0.0238],
         [0.0247, 0.1234, 0.1413, 0.1712, 0.0587, 0.1120],
         [0.0436, 0.0291, 0.0284, 0.0284, 0.0654, 0.0284],
         [0.0185, 0.0793, 0.1298, 0.0080, 0.0131, 0.0090],
         [0.0996, 0.0431, 0.0488, 0.1303, 0.0767, 0.0830],
         [0.0154, 0.0283, 0.0287, 0.0224, 0.0431, 0.1210],
         [0.0580, 0.0199, 0.0098, 0.0574, 0.0410, 0.0363],
         [0.0332, 0.0344, 0.0344, 0.0934, 0.0456, 0.0706],
         [0.0158, 0.0135, 0.0115, 0.2392, 0.0079, 0.0610]]))

In [17]:
mean2 = mean.std(dim =0)
mean2

tensor([0.0265, 0.0335, 0.0588, 0.0739, 0.0246, 0.0405])

## best valid under lr = 0.0001

In [22]:
best_valid = result[:,0,:]
best_valid.shape, best_valid

(torch.Size([13, 10]),
 tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000],
         [0.9839, 0.9839, 0.9839, 0.9597, 0.9032, 0.8871, 0.9435, 0.9355, 0.9355,
          0.9194],
         [0.9712, 0.9640, 0.9640, 0.9712, 0.9640, 0.9784, 0.9712, 0.9712, 0.9712,
          0.9712],
         [0.9033, 0.8939, 0.9009, 0.7453, 0.8915, 0.9009, 0.9080, 0.8821, 0.9198,
          0.9057],
         [0.8497, 0.8693, 0.8693, 0.8366, 0.8497, 0.8889, 0.8824, 0.8824, 0.8889,
          0.8889],
         [0.8954, 0.9216, 0.9281, 0.9477, 0.9216, 0.9346, 0.9346, 0.8693, 0.9412,
          0.9477],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.8621,
          1.0000],
         [0.8220, 0.8115, 0.8168, 0.8220, 0.8482, 0.8377, 0.8482, 0.8482, 0.8115,
          0.8639],
         [0.3753, 0.4563, 0.4272, 0.4372, 0.3899, 0.5887, 0.6197, 0.4581, 0.6228,
          0.3653],
         [0.9756, 0.9268, 0.9512, 0.9512, 0.9512, 0.9756, 0.9512, 0.

In [23]:
index = best_valid.argmax(dim = 1)
index

tensor([0, 0, 5, 8, 5, 3, 0, 9, 8, 0, 8, 8, 5])

## test

In [8]:

result = torch.zeros((13, 6, 10))
#lrs = [0.05, 0.01, 0.005, 0.001]
lrs = [0.0001, 0.0005,0.001, 0.005, 0.01,0.05]
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(lrs)):
        lr = lrs[j]
        
        for n in range(10):
            seed = n
            if lr == 0.001 :
                setup = f'dataset_{data_name}_lr_{lr}_seed_{seed}_learnable0.001'
            else:
                setup = f'dataset_{data_name}_lr_{lr}_seed_{seed}_learnable'
            model = torch.load(f'./results_variation/pNN_{setup}')
            #model.SetParameter('N', 100)
            
            
            with torch.no_grad():
                for X_valid, y_valid in test_loader:
                    
                    prediction_valid = model(X_valid)
                    #acc_valid = lossfunction(prediction_valid, y_valid)
                    yhat_valid = torch.argmax(prediction_valid.data, 2)
                
                    yy_valid = y_valid.repeat(prediction_valid.shape[0], 1)
                    valid_correct = torch.sum(yhat_valid == yy_valid.data)
                    acc_valid = valid_correct / (y_valid.numel() * prediction_valid.shape[0])
                    result[i,j,n] = acc_valid
                

#a = {'acc_valid': result}
#with open('acc_valid_variation.txt', 'wb') as file:
    #pickle.dump(a, file)

In [13]:
result

tensor([[[1.0000, 1.0000, 0.9200, 1.0000, 0.7600, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000]],

        [[0.9127, 0.8413, 0.9524, 0.9206, 0.8254, 0.8175, 0.9127, 0.8254,
          0.9048, 0.8333],
         [0.9524, 0.9048, 0.9365, 0.8968, 0.9048, 0.9444, 0.8968, 0.8492,
          0.9444, 0.8254],
         [0.8810, 0.9286, 0.9048, 0.8889, 0.9365, 0.8968, 0.8968, 0.9048,
          0.9048, 0.8810],
         [0.8968, 0.9127, 0.9286, 0.9286, 0.9206, 0.9048, 0.9127, 0.8889,
          0.8413, 0.8810],
         [0.8889, 0.9206, 0.8333, 0.8968, 0.9048, 0.8889, 0.8889, 0.8254,
          0.841

In [9]:
mean= result.mean(dim=2)
mean.shape, mean

(torch.Size([13, 6]),
 tensor([[1.0000, 0.9680, 1.0000, 1.0000, 1.0000, 1.0000],
         [0.8675, 0.8746, 0.9056, 0.9024, 0.9016, 0.8714],
         [0.9714, 0.9650, 0.9679, 0.9693, 0.9686, 0.9686],
         [0.8756, 0.8315, 0.7796, 0.7751, 0.7925, 0.7925],
         [0.9084, 0.8909, 0.8182, 0.8656, 0.8273, 0.8591],
         [0.9032, 0.8877, 0.8175, 0.7968, 0.8097, 0.8649],
         [0.9032, 0.9613, 0.9323, 0.9516, 0.9097, 0.9516],
         [0.8073, 0.7990, 0.7886, 0.7876, 0.7850, 0.7938],
         [0.4669, 0.5419, 0.2191, 0.2050, 0.1897, 0.2225],
         [0.9023, 0.9070, 0.8093, 0.8628, 0.8279, 0.8512],
         [0.8943, 0.9312, 0.7380, 0.7651, 0.8010, 0.8115],
         [0.8190, 0.8000, 0.7778, 0.6714, 0.7048, 0.7127],
         [0.7984, 0.8111, 0.7556, 0.6937, 0.7810, 0.7794]]))

In [10]:
mean1 = result.std(dim=2)
mean1

tensor([[0.0000, 0.0773, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0290, 0.0504, 0.0421, 0.0183, 0.0265, 0.0360],
        [0.0000, 0.0071, 0.0051, 0.0048, 0.0037, 0.0037],
        [0.0407, 0.0333, 0.0343, 0.0268, 0.0422, 0.0422],
        [0.0429, 0.0468, 0.1744, 0.0516, 0.1230, 0.0507],
        [0.0215, 0.0594, 0.1222, 0.1492, 0.1126, 0.0468],
        [0.1075, 0.0136, 0.1014, 0.0228, 0.1029, 0.0170],
        [0.0133, 0.0106, 0.0117, 0.0073, 0.0177, 0.0163],
        [0.1183, 0.1096, 0.1541, 0.1272, 0.1045, 0.1120],
        [0.0147, 0.0110, 0.1149, 0.0319, 0.1124, 0.0480],
        [0.0790, 0.0551, 0.2751, 0.0888, 0.1313, 0.1268],
        [0.0171, 0.0468, 0.0785, 0.0599, 0.0911, 0.0845],
        [0.0168, 0.0190, 0.1160, 0.2157, 0.0490, 0.0526]])

In [11]:
mean2 = mean.mean(dim= 0)
mean2

tensor([0.8552, 0.8592, 0.7930, 0.7882, 0.7922, 0.8061])

In [12]:
std = mean1.mean(dim=0)
std

tensor([0.0385, 0.0415, 0.0946, 0.0619, 0.0705, 0.0490])

In [18]:
for i in range(13):
    s = ""
    for j in range(6):
        s = s+"& $" +str(result1[i,j].numpy().round(3)) +"\pm "+str(mean[i,j].numpy().round(3)) +"$ "
    print(s)

& $1.0\pm 0.0$ & $1.0\pm 0.0$ & $0.922\pm 0.165$ & $1.0\pm 0.0$ & $1.0\pm 0.0$ & $1.0\pm 0.0$ 
& $0.944\pm 0.034$ & $0.965\pm 0.007$ & $0.96\pm 0.0$ & $0.976\pm 0.012$ & $0.969\pm 0.011$ & $0.967\pm 0.008$ 
& $0.97\pm 0.005$ & $0.965\pm 0.009$ & $0.96\pm 0.005$ & $0.97\pm 0.006$ & $0.955\pm 0.009$ & $0.966\pm 0.006$ 
& $0.885\pm 0.05$ & $0.854\pm 0.025$ & $0.853\pm 0.041$ & $0.76\pm 0.033$ & $0.796\pm 0.047$ & $0.812\pm 0.053$ 
& $0.871\pm 0.019$ & $0.825\pm 0.026$ & $0.806\pm 0.126$ & $0.819\pm 0.037$ & $0.841\pm 0.039$ & $0.809\pm 0.024$ 
& $0.924\pm 0.025$ & $0.866\pm 0.123$ & $0.842\pm 0.141$ & $0.803\pm 0.171$ & $0.866\pm 0.059$ & $0.844\pm 0.112$ 
& $0.986\pm 0.044$ & $0.986\pm 0.029$ & $0.955\pm 0.028$ & $0.976\pm 0.028$ & $0.91\pm 0.065$ & $0.941\pm 0.028$ 
& $0.833\pm 0.019$ & $0.791\pm 0.079$ & $0.666\pm 0.13$ & $0.827\pm 0.008$ & $0.818\pm 0.013$ & $0.825\pm 0.009$ 
& $0.474\pm 0.1$ & $0.478\pm 0.043$ & $0.424\pm 0.049$ & $0.209\pm 0.13$ & $0.366\pm 0.077$ & $0.37\pm 0.083$ 

In [19]:
s = ""
for j in range(6):
    s = s+"& $" +str(result2[j].numpy().round(3)) +"\pm "+str(mean2[j].numpy().round(3)) +"$ "
print(s)

& $0.883\pm 0.027$ & $0.855\pm 0.034$ & $0.827\pm 0.059$ & $0.812\pm 0.074$ & $0.826\pm 0.025$ & $0.837\pm 0.04$ 


## learnable, N = 100. 

In [81]:
def basic(prediction, y, *args, **kwargs):
    act, idx = torch.max(prediction, dim=1)
    corrects = (y.view(-1) == idx)
    return corrects.float().sum().item() / y.numel()

In [82]:
def BASIC_variation(nn, x, y, *args, **kwargs):
    prediction = nn(x)
    N = prediction.shape[0]
    accs = []
    for n in range(N):
        accs.append(basic(prediction[n,:,:], y))
    return np.mean(accs), np.std(accs)

In [83]:
index = torch.tensor([0, 0, 5, 8, 5, 3, 0, 9, 8, 0, 8, 8, 5])

In [88]:
results_test1= torch.zeros((13,1,3,2))
#result = torch.zeros((13, 6, 10))
lrs = [0.0001]
epsilons = [0.05, 0.1, 0.3]
#lrs = [0.0001, 0.0005,0.001, 0.005, 0.01,0.05]
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(lrs)):
        lr = lrs[j]
          
        setup = f'dataset_{data_name}_lr_{lr}_seed_{index[i]}_learnable1'
        model = torch.load(f'./results_variation/pNN_{setup}')
        model.SetParameter('N', 50)
        for k in range(len(epsilons)):
            epsilon = epsilons[k]
            model.SetParameter('epsilon', epsilon)
            
            
            with torch.no_grad():
                for X_test, y_test in test_loader:
                    #print(setup)
                    BASIC_variation(model, X_test, y_test)
                    acc, std = BASIC_variation(model, X_test, y_test)
                    results_test1[i,j,k,0], results_test1[i,j,k,1] = acc, std
                

#a = {'acc_valid': result}
#with open('acc_valid_variation.txt', 'wb') as file:
    #pickle.dump(a, file)

In [59]:
results_test1[:,0,0,0], results_test1[:,0,0,1]

(tensor([1.0000, 0.9107, 0.9351, 0.7512, 0.8136, 0.8401, 0.8681, 0.6897, 0.3410,
         0.8821, 0.6332, 0.7206, 0.6416]),
 tensor([0.0000, 0.0166, 0.0823, 0.2177, 0.1581, 0.1026, 0.1114, 0.1184, 0.1195,
         0.0595, 0.2158, 0.0866, 0.1637]))

In [60]:
results_test1[:,0,0,0].mean(), results_test1[:,0,0,1].mean()

(tensor(0.7713), tensor(0.1117))

In [61]:
results_test1[:,0,1,0], results_test1[:,0,1,1]

(tensor([0.9880, 0.8975, 0.8117, 0.6103, 0.6325, 0.6964, 0.7735, 0.6267, 0.2471,
         0.7395, 0.5695, 0.6744, 0.5243]),
 tensor([0.0447, 0.0447, 0.2334, 0.2841, 0.2307, 0.2128, 0.1438, 0.1147, 0.1226,
         0.2048, 0.1733, 0.0787, 0.1991]))

In [62]:
results_test1[:,0,1,0].mean(), results_test1[:,0,1,1].mean()

(tensor(0.6763), tensor(0.1606))

In [63]:
results_test1[:,0,2,0], results_test1[:,0,2,1]

(tensor([0.7776, 0.6898, 0.6139, 0.2662, 0.5310, 0.5059, 0.5303, 0.5557, 0.1395,
         0.4879, 0.5267, 0.6170, 0.3867]),
 tensor([0.2366, 0.2488, 0.2330, 0.2800, 0.2266, 0.2194, 0.1819, 0.0681, 0.0698,
         0.2262, 0.1278, 0.0841, 0.1695]))

In [64]:
results_test1[:,0,0].mean(), results_test1[:,0,0].std()

(tensor(0.4415), tensor(0.3599))

## epsilon= 0, N = 1

In [92]:
results_test1= torch.zeros((13,1,2))
#result = torch.zeros((13, 6, 10))
lrs = [0.0001]
#epsilons = [0.05, 0.1, 0.3]
#lrs = [0.0001, 0.0005,0.001, 0.005, 0.01,0.05]
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(lrs)):
        lr = lrs[j]
          
        setup = f'dataset_{data_name}_lr_{lr}_seed_{index[i]}_learnable1'
        model = torch.load(f'./results_variation/pNN_{setup}')
        #model.SetParameter('N', 50)
        
            
            
        with torch.no_grad():
            for X_test, y_test in test_loader:
                    #print(setup)
                BASIC_variation(model, X_test, y_test)
                acc, std = BASIC_variation(model, X_test, y_test)
                results_test1[i,j,0], results_test1[i,j,1] = acc, std
                

#a = {'acc_valid': result}
#with open('acc_valid_variation.txt', 'wb') as file:
    #pickle.dump(a, file)

In [93]:
results_test1[:,0,0],results_test1[:,0,1]

(tensor([1.0000, 0.9365, 0.9714, 0.8991, 0.9351, 0.9221, 0.9677, 0.8083, 0.6198,
         0.9070, 1.0000, 0.8254, 0.7937]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

## nonlearnable1

In [85]:
index2 =torch.tensor([0, 0, 2, 4, 3, 6, 1, 2, 6, 1, 0, 0, 3])

In [89]:
results_test= torch.zeros((13,1,3,2))
#result = torch.zeros((13, 6, 10))
lrs = [0]
epsilons = [0.05, 0.1, 0.3]
#lrs = [0.0001, 0.0005,0.001, 0.005, 0.01,0.05]
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(lrs)):
        lr = lrs[j]
          
        setup = f'dataset_{data_name}_seed_{index2[i]}_nonlearnable1'
        model = torch.load(f'./results_variation/pNN_{setup}')
        model.SetParameter('N', 50)
        for k in range(len(epsilons)):
            epsilon = epsilons[k]
            model.SetParameter('epsilon', epsilon)
            
            
            with torch.no_grad():
                for X_test, y_test in test_loader:
                   
                    #BASIC_variation(model, X_test, y_test)
                    acc, std = BASIC_variation(model, X_test, y_test)
                    results_test[i,j,k,0], results_test[i,j,k,1] = acc, std

## N = 1

In [95]:
results_test= torch.zeros((13,1,2))
#result = torch.zeros((13, 6, 10))
lrs = [0]
#epsilons = [0.05, 0.1, 0.3]
#lrs = [0.0001, 0.0005,0.001, 0.005, 0.01,0.05]
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(lrs)):
        lr = lrs[j]
          
        setup = f'dataset_{data_name}_seed_{index2[i]}_nonlearnable1'
        model = torch.load(f'./results_variation/pNN_{setup}')
        #model.SetParameter('N', 50)
        
        #model.SetParameter('epsilon', epsilon)
            
            
        with torch.no_grad():
            for X_test, y_test in test_loader:
                    #print(setup)
                BASIC_variation(model, X_test, y_test)
                acc, std = BASIC_variation(model, X_test, y_test)
                results_test[i,j,0], results_test[i,j,1] = acc, std
                

#a = {'acc_valid': result}
#with open('acc_valid_variation.txt', 'wb') as file:
    #pickle.dump(a, file)

In [96]:
results_test[:,j,0], results_test[:,j,1]

(tensor([1.0000, 0.8889, 0.9500, 0.8944, 0.9481, 0.8961, 0.9677, 0.8135, 0.5825,
         0.8837, 0.7396, 0.8095, 0.8413]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [67]:
results_test[:,0,0,0], results_test[:,0,0,1]

(tensor([0.9648, 0.9006, 0.8527, 0.8094, 0.8459, 0.7899, 0.8671, 0.6359, 0.3642,
         0.8209, 0.6961, 0.7152, 0.6495]),
 tensor([0.0788, 0.0243, 0.1805, 0.1217, 0.1037, 0.1138, 0.1162, 0.1437, 0.1159,
         0.1401, 0.0360, 0.0812, 0.1556]))

In [68]:
results_test[:,0,0,0].mean(), results_test[:,0,0,1].mean()

(tensor(0.7625), tensor(0.1086))

In [69]:
results_test[:,0,1,0], results_test[:,0,1,1]

(tensor([0.8764, 0.8663, 0.7066, 0.7227, 0.6901, 0.7064, 0.7168, 0.5602, 0.2511,
         0.6247, 0.6490, 0.6565, 0.5176]),
 tensor([0.1721, 0.1029, 0.2633, 0.2070, 0.1759, 0.1335, 0.1876, 0.1156, 0.1126,
         0.2492, 0.0733, 0.1237, 0.1899]))

In [70]:
results_test[:,0,1,0].mean(), results_test[:,0,1,1].mean()

(tensor(0.6573), tensor(0.1620))

In [71]:
results_test[:,0,2,0], results_test[:,0,2,1]

(tensor([0.6212, 0.6467, 0.5849, 0.5465, 0.4557, 0.5355, 0.4968, 0.5066, 0.1661,
         0.3949, 0.5612, 0.5759, 0.4146]),
 tensor([0.2214, 0.2734, 0.2601, 0.2976, 0.1831, 0.2042, 0.1905, 0.0681, 0.0841,
         0.2234, 0.1165, 0.1334, 0.1683]))

In [72]:
results_test[:,0,2,0].mean(), results_test[:,0,2,1].mean()

(tensor(0.5005), tensor(0.1865))

In [90]:
for j in range(3):
    for i in range(13):
        s = ""
    
        s = s+"& $" +str(results_test[i,0,j,0].numpy().round(3)) +"\pm "+str(results_test[i,0,j,1].numpy().round(3)) +"$ " + "& $" +str(results_test1[i,0,j,0].numpy().round(3)) +"\pm "+str(results_test1[i,0,j,1].numpy().round(3))+"$"
        #s = str(datasets[i][:-2]) + s + "\\" + "\\"
        s = datasets[i][8:-2] + s + "\\" + "\\"
        print(s)
        print("\hline")
    temp = results_test[:,0,j,0].numpy()
    Temp = results_test[:,0,j,1].numpy()
    temp1 = results_test1[:,0,j,0].numpy()
    Temp1 = results_test1[:,0,j,1].numpy()
    print("& $" +str(np.mean(temp).round(3))+"\pm "+str(np.mean(Temp).round(3))+"$"+"& $" +str(np.mean(temp1).round(3))+"\pm "+str(np.mean(Temp1).round(3))+"$")
    print("#############################################")

acuteinflammation& $0.974\pm 0.066$ & $1.0\pm 0.0$\\
\hline
balancescale& $0.904\pm 0.013$ & $0.909\pm 0.019$\\
\hline
breastcancerwisc& $0.9\pm 0.137$ & $0.898\pm 0.144$\\
\hline
cardiotocography3clases& $0.822\pm 0.08$ & $0.711\pm 0.207$\\
\hline
energyy1& $0.792\pm 0.135$ & $0.847\pm 0.106$\\
\hline
energyy2& $0.772\pm 0.114$ & $0.843\pm 0.105$\\
\hline
iris& $0.865\pm 0.106$ & $0.847\pm 0.116$\\
\hline
mammographic& $0.631\pm 0.133$ & $0.689\pm 0.124$\\
\hline
Pendigits& $0.368\pm 0.101$ & $0.36\pm 0.118$\\
\hline
seeds& $0.817\pm 0.139$ & $0.877\pm 0.08$\\
\hline
tictactoe& $0.686\pm 0.048$ & $0.673\pm 0.191$\\
\hline
vertebralcolumn2clases& $0.711\pm 0.071$ & $0.71\pm 0.082$\\
\hline
vertebralcolumn3clases& $0.656\pm 0.138$ & $0.686\pm 0.147$\\
\hline
& $0.761\pm 0.099$& $0.773\pm 0.111$
#############################################
acuteinflammation& $0.848\pm 0.164$ & $0.982\pm 0.059$\\
\hline
balancescale& $0.882\pm 0.065$ & $0.868\pm 0.109$\\
\hline
breastcancerwisc& $0.719\p

In [98]:
for j in range(1):
    for i in range(13):
        s = ""
    
        s = s+"& $" +str(results_test[i,j,0].numpy().round(3)) +"$ " + "& $" +str(results_test1[i,j,0].numpy().round(3)) +"$"
        #s = str(datasets[i][:-2]) + s + "\\" + "\\"
        s = datasets[i][8:-2] + s + "\\" + "\\"
        print(s)
        print("\hline")
    temp = results_test[:,j,0].numpy()
    #Temp = results_test[:,j,1].numpy()
    temp1 = results_test1[:,j,0].numpy()
    #Temp1 = results_test1[:,j,1].numpy()
    print("& $" +str(np.mean(temp).round(3))+"$"+"& $" +str(np.mean(temp1).round(3))+"$")
    print("#############################################")

acuteinflammation& $1.0$ & $1.0$\\
\hline
balancescale& $0.889$ & $0.937$\\
\hline
breastcancerwisc& $0.95$ & $0.971$\\
\hline
cardiotocography3clases& $0.894$ & $0.899$\\
\hline
energyy1& $0.948$ & $0.935$\\
\hline
energyy2& $0.896$ & $0.922$\\
\hline
iris& $0.968$ & $0.968$\\
\hline
mammographic& $0.813$ & $0.808$\\
\hline
Pendigits& $0.583$ & $0.62$\\
\hline
seeds& $0.884$ & $0.907$\\
\hline
tictactoe& $0.74$ & $1.0$\\
\hline
vertebralcolumn2clases& $0.81$ & $0.825$\\
\hline
vertebralcolumn3clases& $0.841$ & $0.794$\\
\hline
& $0.863$& $0.891$
#############################################


In [142]:
-3// 5

-1